Readme - I wrote this code on Kaggle notebook to use GPU's to ran it asotherwise it would take a lifetime to run it over CPU. So kindly run it over Kaggle. It's giving close to 99.5% percent. Here is the public notebook link - https://www.kaggle.com/anuragsharma1097/distracted-driver-detection/edit

In [ ]:
### Importing all the required libraries

import cv2
from glob import glob
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Activation, MaxPooling2D, Dropout, Flatten
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.optimizers import RMSprop
import numpy as np

In [ ]:
images = []
labels = []

def get_data(clazz):
    """
    This function is used to match all the image files on Kaggle and load all the images and labels from all the files
    and store them in separate lists of images and lables.
    "param clazz": This param corresponds to labels i.e. 0, 1, 2,3 ...,8, 9 to match the filename and load the images
                   from it.
    """
    files = glob(os.path.join('..', 'input','state-farm-distracted-driver-detection','imgs', 'train', 'c' + str(clazz), '*.jpg'))
    for file in files:
        img = cv2.imread(file) # Reading the images
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Converting them to greyscale from RGB.
        images.append(cv2.resize(img, (225,225))) # Resizing the image to 225,225 pixels
        labels.append(clazz)

In [ ]:
# Calling the get_date function for all the labels 0-9, to load all the files for all the classes

for clazz in range(10):
    get_data(clazz)

In [ ]:
# Splitting the dataset into train and test data with train_size=0.75 and test_size=0.25
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.25, random_state=0)

In [ ]:
# Converting the labels to one-hot encoding using keras.to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Converting training and testing input  to numpy arrays
x_train = np.asarray(x_train)
x_test = np.asarray(x_test)

# changing the image dimensions to (225, 225, 1) as we only need one channel.
x_test = np.expand_dims(x_test, axis=3)
x_train = np.expand_dims(x_train, axis=3)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
# Exploring the dataset by plotting the iput images on a (8, 8) grid just ot get a jist of how data looks.

fig = plt.figure(figsize=(8,8))
for i in range(64):
    ax = fig.add_subplot(8,8,i+1)
    ax.imshow(x_train[i], cmap=plt.cm.bone)
plt.show()

In [ ]:
# Coreating the CNN model

model = Sequential()

# Adding two convolutional layers with 32 units each, 'same' padding, 'elu' activation function.
model.add(Conv2D(32 ,kernel_size=(3, 3), padding='same', input_shape=(225, 225, 1), strides=(1, 1), activation='elu'))
model.add(Conv2D(32, (3, 3), activation='elu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) # Using pooling layer to decrease the image size to reduce computation
model.add(Dropout(0.25)) # Using a dropout layer with 25% probability

# Adding two convolutional layers with 64 units each, 'same' padding, 'elu' activation function.
model.add(Conv2D(64 ,kernel_size=(3, 3), padding='same', activation='elu'))
model.add(Conv2D(64, (3, 3), activation='elu'))
model.add(MaxPooling2D(pool_size=(2, 2))) # Using pooling layer to decrease the image size to reduce computation
model.add(Dropout(0.25)) # Using a dropout layer with 25% probability

# Adding two convolutional layers with 128 units each, 'same' padding, 'elu' activation function.
model.add(Conv2D(128 ,kernel_size=(3, 3), padding='same', activation='elu'))
model.add(Conv2D(128, (3, 3), activation='elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))# Using pooling layer to decrease the image size to reduce computation
model.add(Dropout(0.25)) # Using a dropout layer with 25% probability

model.add(Flatten()) # Flattening the image array
model.add(Dense(256, activation='elu')) # Dense layer with 256 units and 'elu' activation function
model.add(Dropout(0.25)) # Using a dropout layer with 25% probability

# Using the output layer with 10 units as we have 10 labels and 'softmax' activation function
model.add(Dense(10, activation='softmax')) 

In [ ]:
model.summary() # Printing the model summary

In [ ]:
# Using the RMSprop with learning rate of '0.0001' and '1e-6' decay. Tried using Adam optimizer as well but it 
# reduced the accuracy to 98.3% percent from 99.5%

opt = RMSprop(lr=0.0001, decay=1e-6)

# Compiling the the model using the 'categorical_crossentropy' loss fucntion and accuracy as the metrics
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
# Fitting the model with 10 epochs and batch_size of 50
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=50)

In [ ]:
# Printing the accuracy which is close to 99.5%. I'll add the 

_, acc = model.evaluate(x_test, y_test, verbose=10)
print(acc)